In [25]:
!pip install -q pyomo
!apt-get install -y -qq coinor-cbc

## 1)

Let $x_{ij}$ denote placement of facility $i$ in a location $j$. \
 $x_{ij} = 1$ if facility $i$ is placed at location $j$, and  $x_{ij} = 0$ otherwise \

Let C be the matrix:\
\begin{equation}
   \begin{matrix} 
22&22&19&22&22&24&18&17&21&23&19&21\\
18&12&18&19&22&21&17&20&16&17&17&24\\
19&21&17&19&17&19&24&16&18&16&20&24\\
18&24&20&21&21&21&22&19&18&21&23&23\\
23&20&19&18&20&23&22&25&24&19&21&23\\
24&16&15&16&24&21&23&21&20&20&22&19\\
22&22&21&23&18&17&16&19&24&20&20&23\\
23&17&17&17&22&24&23&20&22&19&23&21\\
21&22&21&24&20&23&19&18&23&15&25&21\\
19&19&18&19&26&24&19&17&18&20&21&20\\
20&22&23&20&22&20&20&19&17&19&20&24\\ 
21&25&21&19&21&19&18&16&22&24&25&24
   \end{matrix} 
\end{equation}  

Let $c_{ij}$ denote element of row $i$ and column $j$ of C. \

## Model

Decision variables: $x_{i,j} \qquad i = 1,2,3...12 \ ,\ j=1,2,3...12$

Objective: $\ minimize \sum_{i=1}^{12} \sum_{j=1}^{12} c_{ij}x_{ij}$

Subject to:
\begin{align}
\sum_{j=1}^{12} x_{1,j} &= 1 \nonumber \\
\sum_{j=1}^{12} x_{2,j} &= 1 \nonumber \\
\sum_{j=1}^{12} x_{3,j} &= 1 \nonumber \\
\sum_{j=1}^{12} x_{4,j} &= 1 \nonumber \\
\sum_{j=1}^{12} x_{5,j} &= 1 \nonumber \\
\sum_{j=1}^{12} x_{6,j} &= 1 \nonumber \\
\sum_{j=1}^{12} x_{7,j} &= 1 \nonumber \\
\sum_{j=1}^{12} x_{8,j} &= 1 \nonumber \\
\sum_{j=1}^{12} x_{9,j} &= 1 \nonumber \\
\sum_{j=1}^{12} x_{10,j} &= 1 \nonumber \\
\sum_{j=1}^{12} x_{11,j} &= 1 \nonumber \\
\sum_{j=1}^{12} x_{12,j} &= 1 \nonumber \\
\sum_{i=1}^{12} x_{i,1} &= 1 \nonumber \\
\sum_{i=1}^{12} x_{i,2} &= 1 \nonumber \\
\sum_{i=1}^{12} x_{i,3} &= 1 \nonumber \\
\sum_{i=1}^{12} x_{i,4} &= 1 \nonumber \\
\sum_{i=1}^{12} x_{i,5} &= 1 \nonumber \\
\sum_{i=1}^{12} x_{i,6} &= 1 \nonumber \\
\sum_{i=1}^{12} x_{i,7} &= 1 \nonumber \\
\sum_{i=1}^{12} x_{i,8} &= 1 \nonumber \\
\sum_{i=1}^{12} x_{i,9} &= 1 \nonumber \\
\sum_{i=1}^{12} x_{i,10} &= 1 \nonumber \\
\sum_{i=1}^{12} x_{i,11} &= 1 \nonumber \\
\sum_{i=1}^{12} x_{i,12} &= 1 \nonumber \\
\end{align}
$x_{ij} \in \{0,1\}\ \  \forall \  i \in \{1,2,3,4,5,6,7,8,9,10,11,12\},\ \  \ \forall \ j \in \{1,2,3,4,5,6,7,8,9,10,11,12\}$

In [26]:
from pyomo.environ import *
import numpy as np
import pandas as pd

### 2)

In [27]:
#A general model can be created as follows:
# Assume costarr is cost array loaded from given txt file

def n_assignment_model(n,costarr):
  genmodel = ConcreteModel()
  genmodel.x = Var(np.arange(n),np.arange(n),domain=Binary)
  genmodel.cons = ConstraintList()
  for i in range(n):
    genmodel.cons.add(sum(genmodel.x[i,j] for j in range(n))==1)
    
  for j in range(n):\
    genmodel.cons.add(sum(genmodel.x[i,j] for i in range(n))==1)

  genmodel.cost = Objective(expr = summation(costarr,genmodel.x))  
  return genmodel 

In [28]:
df = np.loadtxt('lab5_ex2.txt')


In [29]:
costs = df[1:,1:]
M = len(costs)

rows = np.arange(M)


### 6)

In [30]:
m2 = n_assignment_model(M,costs)

In [31]:
solver = SolverFactory('cbc')

opt1 = solver.solve(m2)

### 8)

In [32]:
for i in rows:
  for j in rows:
    val = m2.x[i,j]()
    if val == 1:
      print('Facility {} is placed at location {}'.format(i+1,j+1))
      break

Facility 1 is placed at location 7
Facility 2 is placed at location 2
Facility 3 is placed at location 5
Facility 4 is placed at location 1
Facility 5 is placed at location 11
Facility 6 is placed at location 3
Facility 7 is placed at location 6
Facility 8 is placed at location 4
Facility 9 is placed at location 10
Facility 10 is placed at location 12
Facility 11 is placed at location 9
Facility 12 is placed at location 8


In [33]:
print('The optimal cost is',m2.cost())

The optimal cost is 203.0


### 9)

In [34]:
m21 = m2.clone()
m21.x.domain=NonNegativeReals
for i in rows:
  for j in rows:
    m21.x[i,j].setub(1)

In [35]:
opt21=solver.solve(m21)

In [36]:
for i in rows:
  for j in rows:
    val = m21.x[i,j]()
    if val != 0:
      print('Value of x({},{}) is {}'.format(i+1,j+1,val))
print('Value of remaining variables are 0')      

Value of x(1,7) is 1.0
Value of x(2,2) is 1.0
Value of x(3,5) is 1.0
Value of x(4,1) is 1.0
Value of x(5,11) is 1.0
Value of x(6,3) is 1.0
Value of x(7,6) is 1.0
Value of x(8,4) is 1.0
Value of x(9,10) is 1.0
Value of x(10,12) is 1.0
Value of x(11,9) is 1.0
Value of x(12,8) is 1.0


In [37]:
print('The optimal cost is',m21.cost())

The optimal cost is 203.0


Optimal cost is same, and solution is still integer-valued even when we remove the integer constraint from the original program.

This is because an assignment problem is a special case of minimum-cost flow problem(MCFP).
We know that MCFPs give integer optimal solution if the upper and lower bounds on the variables are integers and net flow through a node is an integer.
In the case of this assignment problem without integrality constraints, bounds are 0 and 1 which are integers, and net flows will also be 0,-1 or 1 which are integers\
Hence, we will get optimal solution which is integral

### 11)

In [38]:
costs2 = costs.copy()
costs2 = np.where(costs2>20,costs2*0.875,costs2*1.42)


In [39]:
m22 = m21.clone()
m22.cost.deactivate()
m22.cost2 = Objective(expr=summation(costs2,m22.x))

In [40]:
opt22 = solver.solve(m22)

In [46]:
for i in rows:
  for j in rows:
    val = m22.x[i,j]()
    if val != 0:
      print('Value of x({},{}) is {}'.format(i+1,j+1,val))
print('Value of remaining variables are 0')        

Value of x(1,4) is 1.0
Value of x(2,6) is 1.0
Value of x(3,2) is 1.0
Value of x(4,10) is 1.0
Value of x(5,7) is 1.0
Value of x(6,8) is 1.0
Value of x(7,3) is 1.0
Value of x(8,9) is 1.0
Value of x(9,12) is 1.0
Value of x(10,11) is 1.0
Value of x(11,5) is 1.0
Value of x(12,1) is 1.0
Value of remaining variables are 0


The solution is still integral when the costs have fractional values.
same reason as previous question, since it is MCFP, and even though the costs are fractions, the variable bounds and the flows will still be integers.

### 12)

In [42]:
m23 = m2.clone()

We can modify the model appropriately by setting $\quad x_{2,4}, \ x_{10,6},\  x_{6,11} = 0$\
to ensure facility 2 is not placed at location 4, facility 10 is not placed at loc 6, facility 6 is not placed at location 11


In [43]:


m23.cons.add(m23.x[1,3]==0)
m23.cons.add(m23.x[9,5]==0)
m23.cons.add(m23.x[5,10]==0)


In [44]:
opt23 = solver.solve(m23)


In [47]:
for i in rows:
  for j in rows:
    val = m23.x[i,j]()
    if val != 0:
      print('Value of x({0},{1}) is {2}. (Facility {0} is placed at location {1})'.format(i+1,j+1,val))
print('Value of remaining variables are 0')        
print('The optimal cost is',m23.cost())      

Value of x(1,7) is 1.0. (Facility 1 is placed at location 7)
Value of x(2,2) is 1.0. (Facility 2 is placed at location 2)
Value of x(3,5) is 1.0. (Facility 3 is placed at location 5)
Value of x(4,1) is 1.0. (Facility 4 is placed at location 1)
Value of x(5,11) is 1.0. (Facility 5 is placed at location 11)
Value of x(6,3) is 1.0. (Facility 6 is placed at location 3)
Value of x(7,6) is 1.0. (Facility 7 is placed at location 6)
Value of x(8,4) is 1.0. (Facility 8 is placed at location 4)
Value of x(9,10) is 1.0. (Facility 9 is placed at location 10)
Value of x(10,12) is 1.0. (Facility 10 is placed at location 12)
Value of x(11,9) is 1.0. (Facility 11 is placed at location 9)
Value of x(12,8) is 1.0. (Facility 12 is placed at location 8)
Value of remaining variables are 0
The optimal cost is 203.0


The optimal solution is same as solution of the original problem inspite of adding the location constraints of facilities 2,10 and 6.\
This is because these constraints were anyways satisfied in the original problem.

Alternately, we can also solve the problem by giving very high costs to the restricted facility-location combinations, we will get the same solution.